<a href="https://colab.research.google.com/github/jspinelli21/Assignment-1-Deep-Learning/blob/main/Code_Spinelli1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import time
import os
import datetime

# Your Stack Overflow API Key
API_KEY = "rl_AVoGw4N7mSN1m46uWq4kBjXhT"  # Replace with your actual API key

# Stack Overflow API Base URL
BASE_URL = "https://api.stackexchange.com/2.3"

# Start and end date (YYYY, MM, DD)
START_DATE = datetime.datetime(2010, 1, 1)
END_DATE = datetime.datetime(2025, 3, 13)

# Convert to Unix timestamps
start_timestamp = int(START_DATE.timestamp())
end_timestamp = int(END_DATE.timestamp())

# Step size: 3 months (adjust if needed)
STEP_DAYS = 10  # 3 months
step_seconds = STEP_DAYS * 24 * 60 * 60

all_questions = []
current_start = start_timestamp

while current_start < end_timestamp:
    current_end = min(current_start + step_seconds, end_timestamp)  # Ensure we don’t exceed limits

    print(f"Fetching questions from {datetime.datetime.utcfromtimestamp(current_start)} to {datetime.datetime.utcfromtimestamp(current_end)}")

    params = {
        "order": "desc",
        "sort": "creation",
        "tagged": "nlp",
        "site": "stackoverflow",
        "pagesize": 100,  # Max per request
        "fromdate": current_start,
        "todate": current_end,
        "key": API_KEY
    }

    page = 1  # Track pagination
    while True:
        params["page"] = page
        response = requests.get(f"{BASE_URL}/questions", params=params)

        if response.status_code != 200:
            print("Error:", response.status_code, response.text)
            break

        data = response.json()
        questions = data.get("items", [])

        if not questions:
            break

        # Append questions to the dataset
        all_questions.extend(questions)

        # Handle API rate limits
        if "backoff" in data:
            print(f"API Backoff detected! Sleeping for {data['backoff']} seconds...")
            time.sleep(data["backoff"])

        if not data.get("has_more"):
            break  # No more pages for this date range

        page += 1
        time.sleep(1)  # Avoid hitting API rate limits

    # Move to the next date range
    current_start = current_end

    # Save progress every batch
    save_path = os.path.expanduser("~/Downloads/nlp_questions_with_answers_test3.json")  # Adjust path
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(all_questions, f, indent=4)

    print(f"Saved {len(all_questions)} questions so far...\n")

print(f"Downloaded a total of {len(all_questions)} NLP questions!")
print(f"File saved at: {save_path}")

Fetching questions from 2010-01-01 00:00:00 to 2010-01-11 00:00:00


FileNotFoundError: [Errno 2] No such file or directory: '/root/Downloads/nlp_questions_with_answers_test3.json'

In [ ]:
import json
import pandas as pd

# Upload the JSON file
file_path = "nlp_questions_with_answers_test3.json"  # Adjust the path if necessary

# Read the JSON file
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Convert to DataFrame
df = pd.DataFrame(data)

# Display column names
print("Columns in the dataset:", df.columns.tolist())

# Display the first 5 rows
print("\nFirst 5 rows of the dataset:")
print(df.head())


Columns in the dataset: ['tags', 'owner', 'is_answered', 'view_count', 'answer_count', 'score', 'last_activity_date', 'creation_date', 'question_id', 'content_license', 'link', 'title', 'accepted_answer_id', 'last_edit_date', 'closed_date', 'closed_reason', 'community_owned_date', 'protected_date', 'migrated_from', 'posted_by_collectives']

First 5 rows of the dataset:
                                  tags  \
0                          [java, nlp]   
1  [algorithm, nlp, platform-agnostic]   
2                                [nlp]   
3   [algorithm, twitter, nlp, ranking]   
4   [algorithm, nlp, lexical-analysis]   

                                               owner  is_answered  view_count  \
0  {'account_id': 88546, 'reputation': 471, 'user...         True         767   
1  {'account_id': 18234, 'reputation': 48466, 'us...         True        5502   
2  {'account_id': 79445, 'reputation': 33245, 'us...         True       17073   
3  {'account_id': 1603, 'reputation': 198977, 'us..

In [ ]:
import requests
import json
import time
import os

# Your Stack Overflow API Key
API_KEY = "rl_AVoGw4N7mSN1m46uWq4kBjXhT"  # Replace with your actual API key

# Stack Overflow API Base URL
BASE_URL = "https://api.stackexchange.com/2.3"

# Load previously collected questions
questions_file = os.path.expanduser("nlp_questions_with_answers_test3.json")  # Adjust path if needed

with open(questions_file, "r", encoding="utf-8") as f:
    questions_data = json.load(f)

# Extract accepted answer IDs
accepted_answer_ids = [q["accepted_answer_id"] for q in questions_data if "accepted_answer_id" in q]

# Remove None values (questions without accepted answers)
accepted_answer_ids = [aid for aid in accepted_answer_ids if aid is not None]

# Prepare list to store answer details
all_answers = []

# Fetch answers using accepted answer IDs
for answer_id in accepted_answer_ids:
    print(f"Fetching answer for Accepted Answer ID: {answer_id}")

    answer_url = f"{BASE_URL}/answers/{answer_id}"
    params = {
        "order": "desc",
        "sort": "votes",
        "site": "stackoverflow",
        "key": API_KEY,
        "filter": "withbody"  # Includes the full answer body
    }

    response = requests.get(answer_url, params=params)

    if response.status_code == 200:
        answer_data = response.json().get("items", [])
        if answer_data:
            all_answers.append(answer_data[0])  # Save only the first (accepted) answer
    else:
        print(f"Error fetching answer {answer_id}: {response.status_code}")

    # Respect API rate limits
    time.sleep(2)

# Save collected answers to a JSON file
save_path = os.path.expanduser("~/Downloads/nlp_answers_by_accepted_answer_id2.json")  # Adjust path
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(all_answers, f, indent=4)

print(f"Finished fetching answers! File saved at: {save_path}")


Fetching answer for Accepted Answer ID: 2009546
Fetching answer for Accepted Answer ID: 2006797
Fetching answer for Accepted Answer ID: 1996018
Fetching answer for Accepted Answer ID: 1994896
Fetching answer for Accepted Answer ID: 2066554
Fetching answer for Accepted Answer ID: 2065291
Fetching answer for Accepted Answer ID: 2063534
Fetching answer for Accepted Answer ID: 2089479
Fetching answer for Accepted Answer ID: 2063718
Fetching answer for Accepted Answer ID: 2165936
Fetching answer for Accepted Answer ID: 2164628
Fetching answer for Accepted Answer ID: 2144541
Fetching answer for Accepted Answer ID: 2114956
Fetching answer for Accepted Answer ID: 2107176
Fetching answer for Accepted Answer ID: 4573042
Fetching answer for Accepted Answer ID: 2207216
Fetching answer for Accepted Answer ID: 2181094
Fetching answer for Accepted Answer ID: 2179455
Fetching answer for Accepted Answer ID: 2171542
Fetching answer for Accepted Answer ID: 2292710
Fetching answer for Accepted Answer ID: 

KeyboardInterrupt: 

In [ ]:
import requests
import json
import time
import os

# Your Stack Overflow API Key
API_KEY = "rl_AVoGw4N7mSN1m46uWq4kBjXhT"  # Replace with your actual API key

# Stack Overflow API Base URL
BASE_URL = "https://api.stackexchange.com/2.3"

# Load previously collected questions
questions_file = os.path.expanduser("nlp_questions_with_answers_test3.json")  # Adjust path if needed

with open(questions_file, "r", encoding="utf-8") as f:
    questions_data = json.load(f)

# Extract accepted answer IDs
accepted_answer_ids = [q["accepted_answer_id"] for q in questions_data if "accepted_answer_id" in q]

# Remove None values (questions without accepted answers)
accepted_answer_ids = [aid for aid in accepted_answer_ids if aid is not None]

# Prepare list to store answer details
all_answers = []
save_interval = 10  # Save progress every 10 requests
save_path = os.path.expanduser("~/Downloads/nlp_answers_by_accepted_answer_id2.json")  # Adjust path

# Load previously saved answers (if available)
if os.path.exists(save_path):
    with open(save_path, "r", encoding="utf-8") as f:
        try:
            all_answers = json.load(f)
        except json.JSONDecodeError:
            all_answers = []  # If file is corrupted, start fresh

# Track fetched answer IDs to avoid duplicates
fetched_answer_ids = {ans["answer_id"] for ans in all_answers}

# Fetch answers using accepted answer IDs
for idx, answer_id in enumerate(accepted_answer_ids, start=1):
    if answer_id in fetched_answer_ids:
        continue  # Skip already fetched answers

    print(f"Fetching answer {idx}/{len(accepted_answer_ids)} for Accepted Answer ID: {answer_id}")

    answer_url = f"{BASE_URL}/answers/{answer_id}"
    params = {
        "order": "desc",
        "sort": "votes",
        "site": "stackoverflow",
        "key": API_KEY,
        "filter": "withbody"  # Includes the full answer body
    }

    response = requests.get(answer_url, params=params)

    if response.status_code == 200:
        answer_data = response.json().get("items", [])
        if answer_data:
            all_answers.append(answer_data[0])  # Save only the first (accepted) answer
            fetched_answer_ids.add(answer_id)  # Mark as fetched
    else:
        print(f"Error fetching answer {answer_id}: {response.status_code}")

    # Save progress every `save_interval` iterations
    if idx % save_interval == 0:
        with open(save_path, "w", encoding="utf-8") as f:
            json.dump(all_answers, f, indent=4)
        print(f"Progress saved at iteration {idx}.")

    # Respect API rate limits
    time.sleep(1)

# Final save
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(all_answers, f, indent=4)

print(f"Finished fetching answers! File saved at: {save_path}")


Fetching answer 1/8519 for Accepted Answer ID: 2009546
Fetching answer 2/8519 for Accepted Answer ID: 2006797
Fetching answer 3/8519 for Accepted Answer ID: 1996018
Fetching answer 4/8519 for Accepted Answer ID: 1994896
Fetching answer 5/8519 for Accepted Answer ID: 2066554
Fetching answer 6/8519 for Accepted Answer ID: 2065291
Fetching answer 7/8519 for Accepted Answer ID: 2063534
Fetching answer 8/8519 for Accepted Answer ID: 2089479
Fetching answer 9/8519 for Accepted Answer ID: 2063718
Fetching answer 10/8519 for Accepted Answer ID: 2165936
Progress saved at iteration 10.
Fetching answer 11/8519 for Accepted Answer ID: 2164628
Fetching answer 12/8519 for Accepted Answer ID: 2144541
Fetching answer 13/8519 for Accepted Answer ID: 2114956
Fetching answer 14/8519 for Accepted Answer ID: 2107176
Fetching answer 15/8519 for Accepted Answer ID: 4573042
Fetching answer 16/8519 for Accepted Answer ID: 2207216
Fetching answer 17/8519 for Accepted Answer ID: 2181094
Fetching answer 18/8519 f

ConnectionError: HTTPSConnectionPool(host='api.stackexchange.com', port=443): Max retries exceeded with url: /2.3/answers/55044130?order=desc&sort=votes&site=stackoverflow&key=rl_AVoGw4N7mSN1m46uWq4kBjXhT&filter=withbody (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x135bd5b20>: Failed to resolve 'api.stackexchange.com' ([Errno 8] nodename nor servname provided, or not known)"))

In [ ]:
import requests
import json
import time
import os
import datetime

# Your Stack Overflow API Key
API_KEY = "rl_AVoGw4N7mSN1m46uWq4kBjXhT"

# Stack Overflow API Base URL
BASE_URL = "https://api.stackexchange.com/2.3"

# Start and end date (YYYY, MM, DD)
START_DATE = datetime.datetime(2010, 1, 1)
END_DATE = datetime.datetime(2025, 3, 13)

# Convert to Unix timestamps
start_timestamp = int(START_DATE.timestamp())
end_timestamp = int(END_DATE.timestamp())

# Step size: 10 days (adjust if needed)
STEP_DAYS = 10
step_seconds = STEP_DAYS * 24 * 60 * 60

all_questions = []
current_start = start_timestamp

# Ensure the save directory exists
save_dir = os.path.expanduser("~/Downloads/")
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "nlp_questions_with_answers_test3.json")

while current_start < end_timestamp:
    current_end = min(current_start + step_seconds, end_timestamp)

    print(f"Fetching questions from {datetime.datetime.utcfromtimestamp(current_start)} to {datetime.datetime.utcfromtimestamp(current_end)}")

    params = {
        "order": "desc",
        "sort": "creation",
        "tagged": "nlp",
        "site": "stackoverflow",
        "pagesize": 100,
        "fromdate": current_start,
        "todate": current_end,
        "key": API_KEY
    }

    page = 1
    while True:
        params["page"] = page
        response = requests.get(f"{BASE_URL}/questions", params=params)

        if response.status_code != 200:
            print("Error:", response.status_code, response.text)
            break

        data = response.json()
        questions = data.get("items", [])

        if not questions:
            break

        # Append questions to the dataset
        all_questions.extend(questions)

        # Handle API rate limits
        if "backoff" in data:
            print(f"API Backoff detected! Sleeping for {data['backoff']} seconds...")
            time.sleep(data["backoff"])

        if not data.get("has_more"):
            break

        page += 1
        time.sleep(1)

    # Move to the next date range
    current_start = current_end

    # Save progress every batch
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(all_questions, f, indent=4)

    print(f"Saved {len(all_questions)} questions so far...\n")

print(f"Downloaded a total of {len(all_questions)} NLP questions!")
print(f"File saved at: {save_path}")


Fetching questions from 2010-01-01 00:00:00 to 2010-01-11 00:00:00
Saved 6 questions so far...

Fetching questions from 2010-01-11 00:00:00 to 2010-01-21 00:00:00
Saved 13 questions so far...

Fetching questions from 2010-01-21 00:00:00 to 2010-01-31 00:00:00
Saved 21 questions so far...

Fetching questions from 2010-01-31 00:00:00 to 2010-02-10 00:00:00
Saved 29 questions so far...

Fetching questions from 2010-02-10 00:00:00 to 2010-02-20 00:00:00
Saved 38 questions so far...

Fetching questions from 2010-02-20 00:00:00 to 2010-03-02 00:00:00
Saved 52 questions so far...

Fetching questions from 2010-03-02 00:00:00 to 2010-03-12 00:00:00
Saved 58 questions so far...

Fetching questions from 2010-03-12 00:00:00 to 2010-03-22 00:00:00
Saved 63 questions so far...

Fetching questions from 2010-03-22 00:00:00 to 2010-04-01 00:00:00
Saved 71 questions so far...

Fetching questions from 2010-04-01 00:00:00 to 2010-04-11 00:00:00
Saved 81 questions so far...

Fetching questions from 2010-04

In [ ]:
import requests
import json
import time
import os

# Your Stack Overflow API Key
API_KEY = "rl_AVoGw4N7mSN1m46uWq4kBjXhT"

# Stack Overflow API Base URL
BASE_URL = "https://api.stackexchange.com/2.3"

# Ensure the save directory exists
save_dir = os.path.expanduser("~/Downloads/")
os.makedirs(save_dir, exist_ok=True)

# Load previously collected questions
questions_file = os.path.join(save_dir, "nlp_questions_with_answers_test3.json")

if not os.path.exists(questions_file):
    raise FileNotFoundError(f"Questions file not found: {questions_file}")

with open(questions_file, "r", encoding="utf-8") as f:
    questions_data = json.load(f)

# Extract accepted answer IDs
accepted_answer_ids = [q["accepted_answer_id"] for q in questions_data if "accepted_answer_id" in q]

# Remove None values (questions without accepted answers)
accepted_answer_ids = [aid for aid in accepted_answer_ids if aid is not None]

# Prepare list to store answer details
all_answers = []
save_interval = 10
save_path = os.path.join(save_dir, "nlp_answers_by_accepted_answer_id2.json")

# Load previously saved answers (if available)
if os.path.exists(save_path):
    with open(save_path, "r", encoding="utf-8") as f:
        try:
            all_answers = json.load(f)
        except json.JSONDecodeError:
            all_answers = []

# Track fetched answer IDs to avoid duplicates
fetched_answer_ids = {ans["answer_id"] for ans in all_answers}

# Fetch answers using accepted answer IDs
for idx, answer_id in enumerate(accepted_answer_ids, start=1):
    if answer_id in fetched_answer_ids:
        continue

    print(f"Fetching answer {idx}/{len(accepted_answer_ids)} for Accepted Answer ID: {answer_id}")

    answer_url = f"{BASE_URL}/answers/{answer_id}"
    params = {
        "order": "desc",
        "sort": "votes",
        "site": "stackoverflow",
        "key": API_KEY,
        "filter": "withbody"
    }

    response = requests.get(answer_url, params=params)

    if response.status_code == 200:
        answer_data = response.json().get("items", [])
        if answer_data:
            all_answers.append(answer_data[0])
            fetched_answer_ids.add(answer_id)
    else:
        print(f"Error fetching answer {answer_id}: {response.status_code}")

    # Save progress every `save_interval` iterations
    if idx % save_interval == 0:
        with open(save_path, "w", encoding="utf-8") as f:
            json.dump(all_answers, f, indent=4)
        print(f"Progress saved at iteration {idx}.")

    # Respect API rate limits
    time.sleep(1)

# Final save
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(all_answers, f, indent=4)

print(f"Finished fetching answers! File saved at: {save_path}")



Streaming output truncated to the last 5000 lines.
Fetching answer 3975/8519 for Accepted Answer ID: 50835596
Fetching answer 3976/8519 for Accepted Answer ID: 50775597
Fetching answer 3977/8519 for Accepted Answer ID: 51065067
Fetching answer 3978/8519 for Accepted Answer ID: 51039297
Fetching answer 3979/8519 for Accepted Answer ID: 51014994
Fetching answer 3980/8519 for Accepted Answer ID: 51022249
Progress saved at iteration 3980.
Fetching answer 3981/8519 for Accepted Answer ID: 51011519
Fetching answer 3982/8519 for Accepted Answer ID: 51009893
Fetching answer 3983/8519 for Accepted Answer ID: 50999911
Fetching answer 3984/8519 for Accepted Answer ID: 51036531
Fetching answer 3985/8519 for Accepted Answer ID: 51362072
Fetching answer 3986/8519 for Accepted Answer ID: 50980836
Fetching answer 3987/8519 for Accepted Answer ID: 51063361
Fetching answer 3988/8519 for Accepted Answer ID: 50999285
Fetching answer 3989/8519 for Accepted Answer ID: 50958055
Fetching answer 3990/8519 for 

In [ ]:
# Print a snippet of the saved file
with open(save_path, "r", encoding="utf-8") as f:
    saved_answers = json.load(f)

print(f"\nSnippet of saved answers ({min(3, len(saved_answers))} entries):")
for ans in saved_answers[:3]:
    print(json.dumps(ans, indent=4)[:500])
    print("\n" + "-" * 80 + "\n")


Snippet of saved answers (3 entries):
{
    "owner": {
        "account_id": 75807,
        "reputation": 83350,
        "user_id": 217332,
        "user_type": "registered",
        "accept_rate": 100,
        "profile_image": "https://www.gravatar.com/avatar/c113a6e7b426161ae6e48face10c9593?s=256&d=identicon&r=PG",
        "display_name": "danben",
        "link": "https://stackoverflow.com/users/217332/danben"
    },
    "is_accepted": true,
    "score": 18,
    "last_activity_date": 1262732012,
    "last_edit_date": 1262732012,


--------------------------------------------------------------------------------

{
    "owner": {
        "account_id": 75807,
        "reputation": 83350,
        "user_id": 217332,
        "user_type": "registered",
        "accept_rate": 100,
        "profile_image": "https://www.gravatar.com/avatar/c113a6e7b426161ae6e48face10c9593?s=256&d=identicon&r=PG",
        "display_name": "danben",
        "link": "https://stackoverflow.com/users/217332/danben

In [ ]:
print(json.dumps(saved_answers[:5], indent=4))


[
    {
        "owner": {
            "account_id": 75807,
            "reputation": 83350,
            "user_id": 217332,
            "user_type": "registered",
            "accept_rate": 100,
            "profile_image": "https://www.gravatar.com/avatar/c113a6e7b426161ae6e48face10c9593?s=256&d=identicon&r=PG",
            "display_name": "danben",
            "link": "https://stackoverflow.com/users/217332/danben"
        },
        "is_accepted": true,
        "score": 18,
        "last_activity_date": 1262732012,
        "last_edit_date": 1262732012,
        "creation_date": 1262730013,
        "answer_id": 2009546,
        "question_id": 2009498,
        "content_license": "CC BY-SA 2.5",
        "body": "<p>It's a lot like the way Lucene ranks documents for a given search query.  They use a metric called TF-IDF, where TF is term frequence and idf is inverse document frequency.  The former ranks a document higher the more the query terms appear in that document, and the latter ra